# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
import api_keys

# Use citipy to find the city from its latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lng = []
cities = []

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

# Create a set of random lat and lng combinations
# NOTE: lat_range [0] is -90 and [1] is 90 from above

lat = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lng = np.random.uniform(lng_range[0], lng_range[1], size=1500)
locations = zip(lat, lng)

# Identify nearest city for each lat, lng combination
for lat_lng in locations:
    city_name = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # Add all unique cities to the 'city' 
    if city_name not in cities:
        cities.append(city_name)
        
print(len(cities))

616


In [3]:
# OpenWeatherMap API Key
api_key = "3e05d6377bff3dd3be6f645fdcc57468"

# Base URL for API Call
base_url= 'http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=' + api_key
base_url

'http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=3e05d6377bff3dd3be6f645fdcc57468'

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [4]:
# Use a counter for the script to run through
city_counter = 0

# Open lists for the following columns to hold data
city = []
lats = []
lngs = []
temps = []
humidity = []
clouds = []
winds = []

#Loop through each city, extract required items, and print result
print('------------------------------')
print('City Weather Data Retrieval')
print('------------------------------')

# For/ try loop for city weather tables 
for city in cities:
    try:
        city_weather = (requests.get(base_url + '&q=' + city)).json() 
        city.append(city_weather['name'])
        lats.append(city_weather['coord']['lat'])
        lngs.append(city_weather['coord']['lon'])
        temps.append(city_weather['main']['temp'])
        humidity.append(city_weather['main']['humidity'])
        clouds.append(city_weather['clouds']['all'])
        winds.append(city_weather['wind']['speed'])
        print(f'City number {number} of {len(cities)} complete. | Added {city}')
        
        # Continue the search until the end
        city_counter = city_counter + 1
         
    except (KeyError, IndexError):
        print(f'Missing data in city {city} | Skipping')
        
#print('------------------------------')
#print('End of City Weather Data')
#print('------------------------------')




------------------------------
City Weather Data Retrieval
------------------------------


AttributeError: 'str' object has no attribute 'append'

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression